# Análise comparativa de dados

In [25]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## 1. Leitura do conjunto de dados

In [16]:
df = pd.read_csv('../data/raw/data.csv')
dicionario = pd.read_csv('../data/external/dictionary.csv')
dicionario

,variavel,descricao,tipo,subtipo
0,total_bill,Valor da conta em dólares,quantitativa,continua
1,tip,Valor da gorjeta em dólares,quantitativa,continua
2,sex,Sexo dos clientes,qualitativa,nominal
3,smoker,Se os clientes eram fumantes ou não,qualitativa,nominal
4,day,Dia da semana,qualitativa,ordinal
5,time,Período do dia,qualitativa,ordinal
6,size,Quantidade de pessoas na mesa,quantitativa,discreta


## 2. Limpeza de dados:

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

### 2.1. Tratamento de dados faltantes:

In [17]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

A partir da informação dada pela saída do código acima, é possível perceber que não existem dados faltantes na tabela.

### 2.2. Tratamento de dados discrepantes:

In [30]:
# Separar as variáveis

target_column = 'tip'

# Filtrando as colunas
nominal_columns = (
    dicionario
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    dicionario
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

ordinal_columns = (
    dicionario
    .query("subtipo == 'ordinal' and variavel != @target_column")
    .variavel
    .to_list()
)
discrete_columns = (
    dicionario
    .query("subtipo == 'discreta'and variavel != @target_column")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]
nominal_columns, continuous_columns, ordinal_columns, discrete_columns

(['sex', 'smoker'], ['total_bill', 'tip'], ['day', 'time'], ['size'])

In [43]:
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])
ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

model = LogisticRegression()

# Criando o pipeline completo
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())  
])